In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import analysis

In [2]:
# directory = '/Users/theoares/lqcd/npr_momfrac/examples/'
# file = 'script_output_EXAMPLE.h5'
# f = h5py.File(directory + file, 'r')

In [3]:
momenta = analysis.get_mom_list()

gammas = np.zeros((4,4,4),dtype=complex)
gammas[0] = gammas[0] + np.array([[0,0,0,1j],[0,0,1j,0],[0,-1j,0,0],[-1j,0,0,0]])
gammas[1] = gammas[1] + np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])
gammas[2] = gammas[2] + np.array([[0,0,1j,0],[0,0,0,-1j],[-1j,0,0,0],[0,1j,0,0]])
gammas[3] = gammas[3] + np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]])

In [4]:
cfgbase = 'cl3_16_48_b6p1_m0p2450'
job_num = 13561
base = './output/' + cfgbase + '_' + str(job_num)

props, threepts = analysis.readfile(base)    # C should be a dictionary whose keys are the momentum slices.

ValueError: could not broadcast input array from shape (3,4,3,4) into shape (4,3,4,3)

In [ ]:
propb = analysis.bootstrap(props)
threeptb = analysis.bootstrap(threepts)

In [ ]:
Gamma = analysis.amputate(propb, threeptb)